In [ ]:
import cv2
from ultralytics import YOLO
import random

model = YOLO('../models/first_model.pt')
video_path = 'http://192.168.69.91:8080/video'
cap = cv2.VideoCapture(video_path)

LABEL_COLORS = {}

def get_label_color(label_id):
    if label_id not in LABEL_COLORS:
        LABEL_COLORS[label_id] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return LABEL_COLORS[label_id]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    detections = results[0].boxes

    for box in detections:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        confidence = box.conf[0]
        label_id = int(box.cls[0])
        class_name = model.names[label_id]

        color = get_label_color(label_id)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        text = f'{class_name} {confidence:.2f}'
        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        cv2.circle(frame, (center_x, center_y), 10, color, -1)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import random
import math
import numpy as np
from ultralytics import YOLO
import uvicorn
import nest_asyncio
from collections import defaultdict

nest_asyncio.apply()

app = FastAPI()

video_url = "http://192.168.69.91:8080/video"
cap = cv2.VideoCapture(video_url)

model = YOLO("../models/first_model.pt")
LABEL_COLORS = {}
object_counters = defaultdict(int)

allowed_pairs = {
    ("tank", "friend"),
    ("enemy", "friend"),
    ("enemy", "wall"),
    ("friend", "wall")
}

def get_label_color(label):
    if label not in LABEL_COLORS:
        LABEL_COLORS[label] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return LABEL_COLORS[label]

def generate_frames():
    while True:
        success, frame = cap.read()
        if not success:
            break

        results = model(frame)
        detections = results[0].boxes
        object_counters.clear()
        centers = []

        for box in detections:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            label_id = int(box.cls[0])
            class_name = model.names[label_id]

            object_counters[class_name] += 1
            unique_label = f"{class_name} {object_counters[class_name]}"

            color = get_label_color(class_name)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            text = f"{unique_label} {confidence:.2f}"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            centers.append((center_x, center_y, class_name.lower(), unique_label))
            cv2.circle(frame, (center_x, center_y), 5, color, -1)

            print(f"Detected: {unique_label}")

        for i in range(len(centers)):
            for j in range(i + 1, len(centers)):
                p1_x, p1_y, label1, name1 = centers[i]
                p2_x, p2_y, label2, name2 = centers[j]

                if (label1, label2) in allowed_pairs or (label2, label1) in allowed_pairs:
                    distance_pixels = int(math.dist((p1_x, p1_y), (p2_x, p2_y)))
                    cv2.line(frame, (p1_x, p1_y), (p2_x, p2_y), (255, 255, 0), 2)
                    midpoint = ((p1_x + p2_x) // 2, (p1_y + p2_y) // 2)
                    cv2.putText(frame, f"{distance_pixels}px", midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        _, buffer = cv2.imencode(".jpg", frame)
        frame_bytes = buffer.tobytes()

        yield (b"--frame\r\n"
               b"Content-Type: image/jpeg\r\n\r\n" + frame_bytes + b"\r\n")

@app.get("/video_feed")
def video_feed():
    return StreamingResponse(generate_frames(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [37709]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:58041 - "GET /video_feed?t=1738653229803 HTTP/1.1" 200 OK

0: 384x640 (no detections), 39.3ms
Speed: 1.9ms preprocess, 39.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 40.4ms
Speed: 1.6ms preprocess, 40.4ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.8ms
Speed: 1.6ms preprocess, 38.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 35.2ms
Speed: 1.7ms preprocess, 35.2ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 40.7ms
Speed: 1.9ms preprocess, 40.7ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 35.0ms
Speed: 1.8ms preprocess, 35.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 39.4ms
Speed: 1.5ms preprocess, 39.4ms inference, 0.3ms postprocess per image at shape (1, 3, 3

[ WARN:0@86.124] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 30083.356007 ms
[mjpeg @ 0x1730956f0] overread 8


Note: you may need to restart the kernel to use updated packages.
